<a href="https://colab.research.google.com/github/eriksonas/aider/blob/main/aider_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aider Browser Script Integration in Google Colab
### created by rUv

This notebook demonstrates how to integrate the Aider browser script within Google Colab to streamline your workflow. With Aider, you can easily fetch data, run AI-powered code suggestions, and handle web-based queries directly within Colab, eliminating the need to switch between environments. This seamless integration allows you to focus on development while leveraging the power of AI to automate and assist in your coding tasks.

### Key Features:
- **Seamless Web Integration:** Run browser-based scripts directly within Colab cells.
- **Automated Data Retrieval:** Fetch data from web sources, perform searches, and extract insights in real-time.
- **Enhanced Workflow:** Combine the power of Colab with Aider's automation to optimize your workflow.

### Aider Chat Overview

Aider is an advanced AI-powered coding assistant designed for pair programming in your terminal. It enables developers to collaborate with large language models (LLMs) like GPT-4 and Claude 3.5 to directly edit code within their local Git repositories. The tool supports a variety of programming languages, including Python, JavaScript, TypeScript, PHP, HTML, and CSS, making it versatile for many projects.

#### Key Capabilities:
1. **Interactive Code Editing:** Aider allows you to edit files by interacting with the LLM in real-time. You can request new features, refactor code, fix bugs, or even generate test cases through chat-based interactions.
2. **Git Integration:** The tool automatically commits changes to your Git repository with meaningful commit messages, simplifying version control.
3. **Multi-File Editing:** Aider can handle complex, multi-file edits efficiently by maintaining an internal map of your entire Git repository.
4. **Voice Commands:** In addition to text-based input, Aider supports voice-to-code interactions, allowing you to code hands-free.
5. **LLM Flexibility:** Aider can connect to multiple LLMs, making it adaptable to various AI services.

Aider is particularly well-suited for large codebases and supports both new projects and existing ones. It helps boost productivity by integrating AI coding capabilities directly into your workflow, eliminating the need for copy-pasting between your code editor and an AI assistant.

For more detailed information, you can visit [Aider's GitHub repository](https://github.com/paul-gauthier/aider) or explore their official website at [aider.chat](https://aider.chat).

### Setup Instructions:

1. **Install Required Libraries:**
   - Install the necessary Python packages for Aider, ngrok, and OpenAI

2. **Get Your ngrok API Key:**
   - Sign up at [ngrok.com](https://ngrok.com) and retrieve your API key from the dashboard under "Your Authtoken."
   - Add the API key to Colab's secrets:
     1. Click the folder icon in the left sidebar.
     2. Click on the "Files" tab.
     3. Click on the three dots next to "Files" and select "Add secrets."
     4. Name the secret `NGROK_API_KEY` and paste your API key as the value.

3. **Get Your OpenAI API Key:**
   - Sign up at [platform.openai.com](https://platform.openai.com) and obtain your OpenAI API key from the API section in your dashboard.
   - Add this key to Colab's secrets following the same steps as above:
     1. Name the secret `OPENAI_API_KEY` and paste your OpenAI API key as the value.

4. **Configure Aider Script:**
   - Set up the Aider browser script with the correct inputs, such as your ngrok and OpenAI API keys, and any other necessary configurations.
   
5. **Execute and Fetch Results:**
   - Run the script to start Aider, open a tunnel with ngrok, and access the results via the public URL generated by ngrok.

Follow along with the cells below to begin integrating Aider with your Colab project!

In [ ]:
!pip install aider-chat pyngrok llama-index pyyaml streamlit

In [ ]:
# @title Local Git Configuration

import subprocess

# Initialize a local Git repository
subprocess.run(["git", "init"])

# Set up Git configuration
# Define the variables in the Colab UI using input forms
git_user_name = "Your Name"  # @param {type:"string"}
git_user_email = "your.email@example.com"  # @param {type:"string"}

# Verify the input
print(f"User Name: {git_user_name}")
print(f"User Email: {git_user_email}")

# Configure Git with the provided user name and email
subprocess.run(["git", "config", "--global", "user.name", git_user_name])
subprocess.run(["git", "config", "--global", "user.email", git_user_email])

# Create a README file as an initial commit
subprocess.run(["bash", "-c", 'echo "# Initial Commit" >> README.md'])

# Add the README file to the staging area
subprocess.run(["git", "add", "README.md"])

# Define a commit message as a parameter in the Colab UI
commit_message = "Initial commit"  # @param {type:"string"}

# Commit the README file using the commit message from the UI
subprocess.run(["git", "commit", "-m", commit_message])

# Display Git log to confirm initialization
!git log

In [ ]:
# @title API Keys Configuration

import os
from google.colab import userdata

# Function to retrieve the API key from Colab secrets and set it as an environment variable
def set_api_key_env_var(secret_name, env_var_name):
    try:
        # Retrieve the secret from Colab secrets
        api_key = userdata.get(secret_name)
        if not api_key:
            raise ValueError(f"{secret_name} is missing. Please check your setup.")

        # Set the secret as an environment variable
        os.environ[env_var_name] = api_key
        print(f"{env_var_name} is set as an environment variable.")
    except Exception as e:
        print(f"Error: {e}")

# Retrieve and set the OpenAI API key as an environment variable
set_api_key_env_var('OPENAI_API_KEY', 'OPENAI_API_KEY')

# Verify that the environment variable is set
print("✅ Key Set")


In [ ]:
# @title Start Aider and expose using NGROK

# Import necessary libraries
from pyngrok import ngrok
from google.colab import userdata
import subprocess
import re
from IPython.display import display, HTML
import time

try:
    # Get the ngrok API key from Google Colab secrets
    ngrok_api_key = userdata.get('NGROK_API_KEY')

    if not ngrok_api_key:
        raise ValueError("ngrok API key is missing. Please check your setup.")

    # Set the ngrok auth token
    ngrok.set_auth_token(ngrok_api_key)

    # Start Aider in browser mode using subprocess
    aider_process = subprocess.Popen(['aider', '--browser'], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

    # Capture the external URL output from Aider
    external_url = None
    while True:
        output = aider_process.stdout.readline()
        if output == '' and aider_process.poll() is not None:
            break
        if output:
            print(output.strip())
            # Regex to capture the External URL provided by Aider
            url_match = re.search(r'External URL:\s(https?://\S+)', output)
            if url_match:
                external_url = url_match.group(1)
                print(f"Found External URL: {external_url}")
                break

    if not external_url:
        raise RuntimeError("Aider did not provide an External URL.")

    # Use ngrok to forward the Aider External URL
    public_url = ngrok.connect(addr="8501", proto="http")
    ngrok_url = public_url.public_url

    print(f"Public ngrok URL: {ngrok_url}")
    print("You can now access the Aider dashboard through ngrok.")

    # Create iframe HTML using the correct ngrok URL
    iframe_html = f'<iframe src="{ngrok_url}" height="800" width="100%"></iframe>'
    hyperlink_html = f'<a href="{ngrok_url}" target="_blank">Open Aider Dashboard in New Tab</a>'

    # Display iframe and hyperlink
    display(HTML(f"{hyperlink_html}<br><br>{iframe_html}"))

except Exception as e:
    print(f"An error occurred: {str(e)}")
    print("If the error is related to the API key, make sure you've set up the 'ngrok_api' secret in Google Colab.")
    print("To set up the secret:")
    print("1. Click on the folder icon in the left sidebar.")
    print("2. Click on the 'Files' tab.")
    print("3. Click on the three dots next to 'Files' and select 'Add secrets'.")
    print("4. Add a new secret with the name 'ngrok_api' and paste your ngrok API key as the value.")

# Keep the notebook running
while True:
    time.sleep(60)
